In [1]:
import argparse
import os
import time
import shutil

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
     

import torchvision
import torchvision.transforms as transforms

from models import *


global best_prec
use_gpu = torch.cuda.is_available()
print('=> Building model...')
    
    
    
batch_size = 150
model_name = "resnet20_4bit_quant"
model = resnet20_quant()

print(model)

normalize = transforms.Normalize(mean=[0.491, 0.482, 0.447], std=[0.247, 0.243, 0.262])


train_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomCrop(32, padding=4),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
trainloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True, num_workers=2)


test_dataset = torchvision.datasets.CIFAR10(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.ToTensor(),
        normalize,
    ]))

testloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False, num_workers=2)


print_freq = 100 # every 100 batches, accuracy printed. Here, each batch includes "batch_size" data points
# CIFAR10 has 50,000 training data, and 10,000 validation data.

def train(trainloader, model, criterion, optimizer, epoch):
    batch_time = AverageMeter()
    data_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    model.train()

    end = time.time()
    for i, (input, target) in enumerate(trainloader):
        # measure data loading time
        data_time.update(time.time() - end)

        input, target = input.cuda(), target.cuda()

        # compute output
        output = model(input)
        loss = criterion(output, target)

        # measure accuracy and record loss
        prec = accuracy(output, target)[0]
        losses.update(loss.item(), input.size(0))
        top1.update(prec.item(), input.size(0))

        # compute gradient and do SGD step
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()


        if i % print_freq == 0:
            print('Epoch: [{0}][{1}/{2}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Data {data_time.val:.3f} ({data_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   epoch, i, len(trainloader), batch_time=batch_time,
                   data_time=data_time, loss=losses, top1=top1))

            

def validate(val_loader, model, criterion ):
    batch_time = AverageMeter()
    losses = AverageMeter()
    top1 = AverageMeter()

    # switch to evaluate mode
    model.eval()

    end = time.time()
    with torch.no_grad():
        for i, (input, target) in enumerate(val_loader):
         
            input, target = input.cuda(), target.cuda()

            # compute output
            output = model(input)
            loss = criterion(output, target)

            # measure accuracy and record loss
            prec = accuracy(output, target)[0]
            losses.update(loss.item(), input.size(0))
            top1.update(prec.item(), input.size(0))

            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()

            if i % 66 == 0:  # This line shows how frequently print out the status. e.g., i%5 => every 5 batch, prints out
                print('Test: [{0}/{1}]\t'
                  'Time {batch_time.val:.3f} ({batch_time.avg:.3f})\t'
                  'Loss {loss.val:.4f} ({loss.avg:.4f})\t'
                  'Prec {top1.val:.3f}% ({top1.avg:.3f}%)'.format(
                   i, len(val_loader), batch_time=batch_time, loss=losses,
                   top1=top1))

    print(' * Prec {top1.avg:.3f}% '.format(top1=top1))
    return top1.avg


def accuracy(output, target, topk=(1,)):
    """Computes the precision@k for the specified values of k"""
    maxk = max(topk)
    batch_size = target.size(0)

    _, pred = output.topk(maxk, 1, True, True)
    pred = pred.t()
    correct = pred.eq(target.view(1, -1).expand_as(pred))

    res = []
    for k in topk:
        correct_k = correct[:k].view(-1).float().sum(0)
        res.append(correct_k.mul_(100.0 / batch_size))
    return res


class AverageMeter(object):
    """Computes and stores the average and current value"""
    def __init__(self):
        self.reset()

    def reset(self):
        self.val = 0
        self.avg = 0
        self.sum = 0
        self.count = 0

    def update(self, val, n=1):
        self.val = val
        self.sum += val * n
        self.count += n
        self.avg = self.sum / self.count

        
def save_checkpoint(state, is_best, fdir):
    filepath = os.path.join(fdir, 'checkpoint.pth')
    torch.save(state, filepath)
    if is_best:
        shutil.copyfile(filepath, os.path.join(fdir, 'model_best.pth.tar'))


def adjust_learning_rate(optimizer, epoch):
    """For resnet, the lr starts from 0.1, and is divided by 10 at 80 and 120 epochs"""
    adjust_list = [150, 225]
    if epoch in adjust_list:
        for param_group in optimizer.param_groups:
            param_group['lr'] = param_group['lr'] * 0.1        

#model = nn.DataParallel(model).cuda()
#all_params = checkpoint['state_dict']
#model.load_state_dict(all_params, strict=False)
#criterion = nn.CrossEntropyLoss().cuda()
#validate(testloader, model, criterion)

=> Building model...
ResNet_Cifar(
  (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (conv2): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()
      )
      (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): QuantConv2d(
        16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
        (weight_quant): weight_quantize_fn()

In [ ]:
# This cell won't be given, but students will complete the training

lr = 4e-3
weight_decay = 1e-4
epochs = 100
best_prec = 90.460003

#model = nn.DataParallel(model).cuda()
model.cuda()
criterion = nn.CrossEntropyLoss().cuda()
optimizer = torch.optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=weight_decay)
#cudnn.benchmark = True

if not os.path.exists('result'):
    os.makedirs('result')
fdir = 'result/'+str(model_name)
if not os.path.exists(fdir):
    os.makedirs(fdir)
        

for epoch in range(0, epochs):
    adjust_learning_rate(optimizer, epoch)

    train(trainloader, model, criterion, optimizer, epoch)
    
    # evaluate on test set
    print("Validation starts")
    prec = validate(testloader, model, criterion)

    # remember best precision and save checkpoint
    is_best = prec > best_prec
    best_prec = max(prec,best_prec)
    print('best acc: {:1f}'.format(best_prec))
    save_checkpoint({
        'epoch': epoch + 1,
        'state_dict': model.state_dict(),
        'best_prec': best_prec,
        'optimizer': optimizer.state_dict(),
    }, is_best, fdir)

In [ ]:
# HW

#  1. Train with 4 bits for both weight and activation to achieve >90% accuracy
#  2. Find x_int and w_int for the 2nd convolution layer
#  3. Check the recovered psum has similar value to the un-quantized original psum
#     (such as example 1 in W3S2)

In [4]:
PATH = "result/resnet20_4bit_quant/model_best.pth.tar"
checkpoint = torch.load(PATH)
model.load_state_dict(checkpoint['state_dict'])
device = torch.device("cuda") 

model.cuda()
model.eval()

test_loss = 0
correct = 0

with torch.no_grad():
    for data, target in testloader:
        data, target = data.to(device), target.to(device) # loading to GPU
        output = model(data)
        pred = output.argmax(dim=1, keepdim=True)  
        correct += pred.eq(target.view_as(pred)).sum().item()

test_loss /= len(testloader.dataset)

print('\nTest set: Accuracy: {}/{} ({:.0f}%)\n'.format(
        correct, len(testloader.dataset),
        100. * correct / len(testloader.dataset)))


Test set: Accuracy: 9046/10000 (90%)



In [ ]:
#send an input and grap the value by using prehook like HW3


In [6]:
class SaveOutput:
    def __init__(self):
        self.outputs = []
    def __call__(self, module, module_in):
        self.outputs.append(module_in)
    def clear(self):
        self.outputs = []

save_output = SaveOutput()

for layer in model.modules():
    if isinstance(layer, QuantConv2d):
        print("Hooking QuantConv", layer)
        layer.register_forward_pre_hook(save_output)
        break

dataiter = iter(trainloader)
images, labels = next(dataiter)
images = images.cuda()
out = model(images)

print("Quantconv1 of BasicBlock0 in layer1:", save_output.outputs[0][0].size())

Hooking QuantConv QuantConv2d(
  16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False
  (weight_quant): weight_quantize_fn()
)
Quantconv1 of BasicBlock0 in layer1: torch.Size([150, 16, 32, 32])


In [ ]:
print(model.layer1[0].conv1.weight_q)

In [11]:
print(model.layer1[0].conv1.weight_quant.wgt_alpha)

Parameter containing:
tensor(2.2090, device='cuda:0', requires_grad=True)


In [12]:
w_bit = 4
weight_q = model.layer1[0].conv1.weight_q # quantized value is stored during the training
w_alpha = model.layer1[0].conv1.weight_quant.wgt_alpha   # alpha is defined in your model already. bring it out here  xx.xxx.xxx
w_delta = w_alpha/(2**(w_bit-1)-1)    # delta can be calculated by using alpha and w_bit
weight_int = weight_q/w_delta # w_int can be calculated by weight_q and w_delta
print(weight_int) # you should see clean integer numbers

tensor([[[[-6.0000, -7.0000, -3.0000],
          [-6.0000, -1.0000, -3.0000],
          [ 2.0000,  1.0000,  7.0000]],

         [[-3.0000, -0.0000, -5.0000],
          [-1.0000, -1.0000, -1.0000],
          [ 2.0000,  4.0000,  1.0000]],

         [[-4.0000,  1.0000,  3.0000],
          [ 0.0000,  2.0000,  4.0000],
          [ 3.0000,  3.0000,  4.0000]],

         ...,

         [[-4.0000, -7.0000, -4.0000],
          [-1.0000, -0.0000, -2.0000],
          [ 3.0000,  5.0000, -1.0000]],

         [[-2.0000, -4.0000, -5.0000],
          [-4.0000, -3.0000, -2.0000],
          [-0.0000, -1.0000,  2.0000]],

         [[-1.0000, -3.0000, -4.0000],
          [-1.0000,  1.0000,  1.0000],
          [-0.0000,  1.0000,  3.0000]]],


        [[[ 0.0000, -2.0000,  1.0000],
          [-3.0000, -1.0000,  0.0000],
          [-0.0000, -2.0000, -1.0000]],

         [[ 1.0000,  4.0000,  2.0000],
          [ 1.0000,  4.0000,  1.0000],
          [-2.0000,  1.0000, -1.0000]],

         [[ 1.0000, -0.0000, -1

In [13]:
print(model.layer1[0].conv1.act_alpha)

Parameter containing:
tensor(3.3335, device='cuda:0', requires_grad=True)


In [14]:
x_bit = 4    
x =  save_output.outputs[0][0] # input of the 2nd conv layer by using prehook
x_alpha  = model.layer1[0].conv1.act_alpha
x_delta = x_alpha/(2**(x_bit)-1)

act_quant_fn = act_quantization(x_bit) # define the quantization function
x_q = act_quant_fn(x, x_alpha)         # create the quantized value for x

x_int = x_q/x_delta      ## x_q / x_delta ?
print(x_int) # you should see clean integer numbers 

tensor([[[[ 0.0000, 15.0000,  8.0000,  ...,  8.0000,  8.0000,  7.0000],
          [ 0.0000, 10.0000,  8.0000,  ...,  7.0000,  9.0000,  7.0000],
          [ 0.0000, 10.0000,  8.0000,  ...,  7.0000,  8.0000,  8.0000],
          ...,
          [ 0.0000,  9.0000,  2.0000,  ...,  6.0000,  7.0000, 15.0000],
          [ 0.0000,  6.0000,  7.0000,  ..., 14.0000, 14.0000, 15.0000],
          [ 4.0000,  0.0000,  1.0000,  ...,  0.0000,  0.0000,  0.0000]],

         [[ 5.0000,  1.0000,  3.0000,  ...,  3.0000,  3.0000,  3.0000],
          [ 1.0000,  0.0000,  4.0000,  ...,  3.0000,  3.0000,  4.0000],
          [ 1.0000,  0.0000,  4.0000,  ...,  3.0000,  3.0000,  4.0000],
          ...,
          [ 4.0000,  2.0000,  2.0000,  ...,  5.0000,  5.0000,  6.0000],
          [ 4.0000,  3.0000,  2.0000,  ...,  8.0000,  8.0000, 10.0000],
          [ 2.0000,  2.0000,  2.0000,  ..., 11.0000, 11.0000,  9.0000]],

         [[ 4.0000,  0.0000,  2.0000,  ...,  3.0000,  3.0000,  4.0000],
          [ 5.0000,  0.0000,  

In [15]:
conv_int = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_int.weight = torch.nn.parameter.Parameter(weight_int)

output_int =  conv_int(x_int)    # output_int can be calculated with conv_int and x_int
output_recovered = output_int*w_delta*x_delta  # recover with x_delta and w_delta
print(output_recovered)

tensor([[[[-21.5300, -21.8106, -14.1663,  ..., -17.6729, -16.3404,  -8.6260],
          [-22.7223, -18.0936, -14.3066,  ..., -15.0780, -18.3040, -11.0105],
          [-22.8625, -18.4443, -15.5690,  ..., -15.5690, -18.2339, -10.3793],
          ...,
          [ -9.6079,  -1.1221,  -1.9637,  ..., -26.5093, -25.5976, -13.1845],
          [ -4.8390,  -6.1013,  -1.3325,  ..., -23.9145, -23.8443, -16.1300],
          [-13.4650,  -5.3299,  -4.0676,  ..., -23.4236, -24.1249, -15.7793]],

         [[  0.2104,  21.1794,  14.9378,  ...,  16.6910,  13.3248,  13.1144],
          [  0.7714,  24.9664,  17.1118,  ...,  18.4443,  14.3066,  13.9560],
          [  1.1221,  24.8262,  15.9196,  ...,  16.2001,  12.2027,  13.3949],
          ...,
          [  5.6806,  18.6547,  15.2884,  ...,  32.8912,  32.9613,  30.3665],
          [  4.0676,  15.6391,  13.3248,  ...,  33.1717,  33.7327,  32.7509],
          [  4.6286,  15.4988,  10.9403,  ...,  13.5352,  13.9560,  15.5690]],

         [[ 17.1820,  -1.1221,

In [17]:
#### input floating number / weight quantized version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
conv_ref.weight = model.layer1[0].conv1.weight_q 

output_ref = conv_ref(x)
print(output_ref)    

tensor([[[[-2.4451e+01, -2.3066e+01, -1.4434e+01,  ..., -1.6158e+01,
           -1.6009e+01, -8.5790e+00],
          [-2.6682e+01, -1.8961e+01, -1.4622e+01,  ..., -1.3836e+01,
           -1.7558e+01, -1.0526e+01],
          [-2.7043e+01, -2.0231e+01, -1.5787e+01,  ..., -1.5710e+01,
           -1.8146e+01, -1.0908e+01],
          ...,
          [-9.5443e+00, -5.3082e-01, -4.7271e-01,  ..., -2.6009e+01,
           -2.5484e+01, -1.0704e+01],
          [-5.3958e+00, -5.6898e+00, -2.7097e+00,  ..., -2.7182e+01,
           -2.7244e+01, -1.6944e+01],
          [-1.2607e+01, -5.2239e+00, -5.0512e+00,  ..., -1.3963e+01,
           -1.4245e+01, -7.7965e+00]],

         [[-1.8953e+00,  2.1499e+01,  1.4475e+01,  ...,  1.7200e+01,
            1.3442e+01,  1.2727e+01],
          [-1.0219e+00,  2.5778e+01,  1.7711e+01,  ...,  1.9047e+01,
            1.4862e+01,  1.4323e+01],
          [-9.9471e-01,  2.5182e+01,  1.6320e+01,  ...,  1.6378e+01,
            1.2833e+01,  1.3074e+01],
          ...,
     

In [18]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.0607, device='cuda:0', grad_fn=<MeanBackward0>)


In [19]:
#### input floating number / weight floating number version

conv_ref = torch.nn.Conv2d(in_channels = 64, out_channels=64, kernel_size = 3, bias = False)
weight = model.layer1[0].conv1.weight
mean = weight.data.mean()
std = weight.data.std()
conv_ref.weight = torch.nn.parameter.Parameter(weight.add(-mean).div(std))  ## include the normalization

output_ref = conv_ref(x)
print(output_ref)


tensor([[[[-2.7737e+01, -2.6666e+01, -1.8036e+01,  ..., -1.8594e+01,
           -1.8752e+01, -1.1863e+01],
          [-2.9832e+01, -2.2344e+01, -1.8125e+01,  ..., -1.6069e+01,
           -2.0067e+01, -1.3811e+01],
          [-3.0239e+01, -2.3627e+01, -1.9260e+01,  ..., -1.8118e+01,
           -2.0710e+01, -1.4124e+01],
          ...,
          [-1.1411e+01, -2.9893e+00, -3.4231e+00,  ..., -2.8842e+01,
           -2.8249e+01, -1.4683e+01],
          [-7.6031e+00, -7.3616e+00, -5.0440e+00,  ..., -3.0618e+01,
           -3.0701e+01, -2.1193e+01],
          [-1.5361e+01, -7.9823e+00, -7.8755e+00,  ..., -1.9292e+01,
           -1.9619e+01, -1.3566e+01]],

         [[-1.1015e+00,  2.1975e+01,  1.4413e+01,  ...,  1.7651e+01,
            1.3445e+01,  1.2733e+01],
          [-4.0321e-02,  2.6409e+01,  1.7904e+01,  ...,  1.9452e+01,
            1.4933e+01,  1.4241e+01],
          [ 3.2157e-02,  2.5847e+01,  1.6578e+01,  ...,  1.6680e+01,
            1.2808e+01,  1.2986e+01],
          ...,
     

In [20]:
difference = abs( output_ref - output_recovered )
print(difference.mean())  ## It should be small, e.g.,2.3 in my trainned model

tensor(1.4524, device='cuda:0', grad_fn=<MeanBackward0>)
